<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C4-Web_Hackery/Brute-Forcing_HTML_Form_Authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Brute-Forcing HTML Form Authentication__*

There may come a time in your web hacking career when you need to gain access to a target or, if you're consulting, assess the password strength on an existing web system. It has become increasingly common for web systems to have brute-force protection, whether a captcha, a simple math equation, or a login token that has to be submitted with the request. There are a number of brute forcers that can do the brute-forcing of a POST request to the login script, but in a lot of cases they are not flexible enough to deal with dynamic content or handle simple "are you human?" checks.

We'll create a simple brute forcer that will be useful against WordPress, a popular content management system. Modern WordPress systems include some basic anti brute-force techniques, but still lack account lockouts or strong captchas by default.

In order to brute-force WordPress, our tool needs to meet two requirements: It must retrieve the hidden token from the login form before submitting the password attempt, and it must ensure that we accept cookies in our HTTP session. The remote application sets one or more cookies on first contact, and it will expect the cookies back on a login attempt. In order to parse out the login form values, we'll use the __lxml__ package introduced in "The lxml and BeautifulSoup Packages" on page 74.

Let's get started by having a look at the WordPress login form. You can find this by browsing to _http://<yourtarget>/wp-login.php/_. You can use your browser's tools to "view source" to find the HTML structure. For example, using the Firefox browser, choose Tools -> Web Developer -> Inspector. For the sake of brevity, we've included the relevant form elements only:
```
<form name="loginform" id="loginform" action="http://boodelyboo.com/wordpress/wp-login.php" method="post"> [1]
    <p>
        <label for="user_login">Username or Email Address</label>
        <input type="text" name="log" id="user_login" value="" size="20" /> [2]
    </p>

    <div class="user-pass-wrap">
        <label for="user_pass">Password</label>
        <div class="wp-pwd">
            <input type="password" name="pwd" id="user_pass" value="" size="20" /> [3]
        </div>
    </div>
    <p class="submit">
        <input type="submit" name="wp-submit" id="wp-submit" value="Log in" /> [4]
        <input type="hidden" name="testcookie" value="1" /> [5]
    </p>
</form>
```
Reading through this form, we are privy to some valuable information that we'll need to incorporate into our brute forcer. The first is that the form gets submitted to the _/wp-login.php_ path as an HTTP POST __[1]__. The next elements are all of the fields required in order for the form submission to be successful: __log__ __[2]__ is the variable representing the username, __pwd__ __[3]__ is the variable for the password, __wp-submit__ __[4]__ is the variable for the submit button, and __testcookie__ __[5]__ is the variable for a test cookie. Note that this input is hidden on the form.

The server also sets a couple of cookies when you make contact with the form, and it expects to receive them again when you post the form data. This is the essential piece of the WordPress anti brute-forcing technique. The site checks the cookie against your current user session, so even if you are passing the correct credentials into the login processing script, the authentication will fail if the cookie is not present. When a normal user logs in, the browser automatically includes the cookie. We must duplicate that behavior in the brute-forcing program. We will handle the cookies automatically using the __requests__ library's __Session__ object.

We'll rely on the following request flow in our brute forcer in order to be successful against WordPress:
1. Retrieve the login page and accept all cookies that are returned.
2. Parse out all of the form elements from the HTML.
3. Set the username and/or password to a guess from our dictionary.
4. Send an HTTP POST to the login processing script, including all HTML form fields and our stored cookies.
5. Test to see if we have successfully logged in to the web application.

__Cain & Abel__, a Winzozz-only password recovery tool, includes a large word list for brute-forcing passwords called __cain.txt__. Let's use that file for our password guesses. You can download it directly from Daniel Miessler's GitHub repository __SecLists__.
```
wget https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Software/cain-and-abel.txt
```
By the way, SecLists contains a lot of other word lists, too. We encourage you to browse through the repo for your future hacking projects.

You can see that we are going to be using some new and valuable techniques in this script. We will also mention that you should never test your tooling on a live target, always set up an installation of your target web application with known credentials and verify that you get the desired results.

Let's open a new Python file named __wordpress_killer.py__ and enter the following code:

In [ ]:
from io import BytesIO
from lxml import etree
from queue import Queue

import requests
import sys
import threading
import time

SUCCESS = "Welcome to WordPress!" #[1]
TARGET = "http://boodelyboo,com/wordpress/wp-login.php" #[2]
WORDLIST = "/home/rick/bhp/cain.txt"